# Лабораторная работа №2

Задание: Используя Python и модули requests и re написать скрипт, получающий все адреса подразделов сайта(относительные url) и для каждой из них выполнить поиск адресов электронной почты


In [1]:
# Импортируем модули
import requests
import re

### Комментарии:
Более простым способом вытащить ящики может быть поиск по слову "mailto". Но этот способ учитывает не все ящики, которые на самом деле можно вытащить из страницы.

Например, для сайта http://www.mosigra.ru/ есть строка "// Rating@Mail.ru counter--"
Формально, тут тоже упоминается ящик, но слова mailto здесь нет.

### Эта функция делает "нарезку" на слова

In [2]:
# Функция, которая принимает на вход строку и нарезает ее на слова, состоящие из букв латинского алфавита 
# (кириллица нас не интересут, так как кириллические домены для почты все еще в тесте), цифр 0-9,  
# знаков '.', '-', '_' и '@', которые могут использоваться для имен почтовых ящиков.

# Функция возвращает list, элементами которого являются условно подходящие слова
# и убирает пробелы между словами

# Для удобства весь текст приводится к нижнему регистру

def cut_to_words(a): # передаем строку 
    a = a.lower()
    b = re.split('[^a-z^0-9@_.-]', a)
    
    #убираем из списка "пустые" слова
    x = list()
    for i in b:
        if i != '':
            x.append(i)        
        
    return x

### Эта функция проводит анализ "ящик или нет"

In [3]:
# Функция получает на вход список слов, в которых можно "заподозрить" ящик
# (по сути, список всех выдернутых из текста страницы сайта слов)
# и возвращает список из слов, которые определила как "почтовый ящик"

def mailbox_or_not(x):
    
    # Согласно требованиям при создании ящиков от компании mail.ru (https://help.mail.ru/enmail-help/registration/name),
    # к имени постового ящика могут применятся следующие правила (!Внимание! правило о длине имени здесь игнорируется,
    # так как на других доменах такого ограничения часто нет): 
    # --- любые латинские символы (a-z, A-Z);
    # --- любые цифры (0-9);
    # --- специальные символы: ("-"), ("_") и (".") — данные символы не могут идти друг за другом, 
    #     а также быть первыми и последними символами в имени почтового ящика.

    
    x_new = list()
    j = 0

    for i in x:
        last = len(i) - 1    # номер последней буквы в слове, которое проверяем
        at = False           # наличие в слове ровно 1 буквы @
        #dot = False          # наличие знака . как минимум через один символ после @
    
        # если первый и последние символы не _./@
        if i[0] != '.' and i[0] != '_' and i[0] != '-' and i[last] != '.' and i[last] != '_' and i[last] != '-' and i[0] != '@' and i[last] != '@': 
            for k in range(last):
                if i[k] == '@':
                    at = True
                
                    # продолжаем искать @ после первой найденной в слове
                    t = k + 1
                    while t <= last:    
                        if i[t] == '@':
                            at = False    # если в слове нашлась хотя бы еще одна @, то отбраковываем
                            break
                        t += 1    
                
                    if at == False:
                        break
                    else:
                        # Адрес может выгядеть как xxxxx@xxxx.xx или xxxxx@xxxx.xxx, xxxx@xxxx.xxxx (.it, .ru, .org, .info)
                        # Поэтому отбираем слова, где полсле знака @ сначала идет ХОТЯ БЫ один разрешенный символ, а не точка,
                        # но есть точка, ХОТЯ бы за ДВА символа до конца слова
                                   
                        if i[k + 1] != '.' and i[k + 1] != '_' and i[k + 1] != '-':
                            t = k + 2
                            while t <= last:    
                                if i[t] == '.' and t != (last -1):
                                    #dot = True
                                    x_new.append(i)
                                #    print(x_new[j])
                                    j += 1
                                    break
                                t += 1    
                    
    return x_new


### Эта функция убирает дубликаты

In [4]:
def delete_copy(d):
    d_new = list()
    d_new.append(d[0])

    for k in d:  # для каждого элемента списка    
            # Кажется, в Python 3 нет этой функции:
            # if dict_of_mailadresses.has_key(k) == False:
            # А жаль...
        
            marker = 0        
            for l in d_new:
                if l == k: 
                    marker += 1  # слова нет в новом списке один раз, если marker == 0
                                 # слово есть в новом списке, если marker > 0
            if marker == 0:
                d_new.append(k)

    return d_new

### Эта функция вылавливает ящики со страницы

In [5]:
def looking_for_mails(sitecontent, name):

    # Уберем все лишнее, создав список только из "подозрительных на почту" слов
    sitecontent_by_words = cut_to_words(sitecontent)
    
    # Проанализируем слова из списка на предмет, являются ли они похожими на почтовый ящик. 
    # Получившийся на этом шаге список содержит только имена почтовых ящиков.
    mailboxes = mailbox_or_not(sitecontent_by_words)      
    
    if mailboxes != []:
       
        # Уберем из списка почтовых ящиков все дубликаты
        mailboxes = delete_copy(mailboxes)
    
#        print('\nOn the page of site ', name, ' there are e-mailaddresse:')
#        print('\n'.join(mailboxes))
    else:
        print('No one e-mail address on page ', name, '. Sorry')
    
    return mailboxes

### Эта функция ищет адреса подразделов

In [6]:
# Функция, которая принимает на вход строку и ищет в ней "кодовое выражение" href="/  ,
# и возвращает list, элементами которого являются подходящие выражения

def list_of_subpages(a): # передаем строку 

    # Для удобства весь текст приводится к нижнему регистру
    a = a.lower()
          
    # Пример того, что ищем, - ' href="/allgames/party/'   
        
    # ' href="/' - шаблон 
    # \w* - символы букви цифр после шаблона
    # \S - любой непробельный символ
    # ["] - обрезаем хвост после знака " 
    b = re.findall(r' href="/\w*\S*["]', a)   
    # из выражения выше получаем при выводе что-то такое: href="/face/show/jackal_treasure_island/"

    x = list()

    for line in b:
        if line != ' href="/"':    # это условие пришлось добавить, так как в коде сайта встречаются выражения href="/"
            c = ''
            for k in range(8, len(line)-1):
                c = c + line[k]
            
            extension = False
            
            # чтобы избежать строк вида: min/?b=css/v2/css&amp;f=remodal.css,remodal-default-theme.css,...  
            if len(c) > 50:          
                for t in c:
                    if t == ';' or t == ',':
                        extension = True
                        break
            
            # чтобы избежать картинок и архивов
            h = re.findall(r'.png|.jpg|.tiff|.bmp|.ico|.rar', c)    
            if h != []:
                extension = True
            
            if extension == False:
                x.append(c)        
            
    return x
# Если посмотреть на наш пример и применить эту функцию,
# то на выходе увидим список подобных фраз: face/show/jackal_treasure_island/

### Сложить два списка

In [20]:
def summa(a, b):
    for i in b:
        a.append(i)
    return a    

# Основной блок

In [22]:
# Получить имя сайта, по которому нужна информация
print('Enter sitename, please')

# Введите имя сайта в формате http://... или www...
# Примеры, чтобы не искать: https://geektimes.ru/post/74326/  
# - http://www.mosigra.ru/  
# - http://mlbootcamp.ru/article/tutorial/ 
# - https://pypi.python.org/pypi/requests/2.11.1

#sitename = input()
sitename = 'http://www.mosigra.ru'  #для тестов, чтобы постоянно не вводить имя сайта

# Сделаем минимальную проверку, что введено имя сайта
check_site_name = True

# В этом месте можно добавить более сложне проверки
if sitename[:8] != 'https://' and sitename[:7] != 'http://':
    check_site_name = False
    print('The site name is wrong')
else:
    print('Please, wait a minute...')
    r = requests.get(sitename)
         
    # в эту СТРОКУ запишем содержимое, взятое с исследуемого сайта    
    sitecontent = r.text    
    
    # Выберем с главной страницы ссылки на подразделы:
    subpageslist = list_of_subpages(sitecontent)
    # уберем дубликаты
    pageslist = delete_copy(subpageslist)
    
#    print('\n\tList of subpages:\n (only first ten for example)\n')
#    for u in range (0,10):
#        print(subpageslist[u])
    
    print('List of mails:')
    # Собираем адреса:

    # Сначала возьмем все с главной страницы
    result_list = looking_for_mails(sitecontent, sitename)
    
    # Потом последовательно вытащим адреса с каждой страницы
    place_for_slash = len(sitename)-1
    
    for page in pageslist:
        if sitename[place_for_slash] == '/':
            pagename = sitename + page
        else:
            pagename = sitename + '/' + page
        
        print('Process page ', pagename, '...........')
        r_page = requests.get(pagename)
        pagecontent = r_page.text
        
        temp_result = looking_for_mails(pagecontent, pagename)
        result_list = summa(result_list, temp_result)
                   
    # На этом шаге у нас есть список ВСЕХ адресов, со всех страниц. НО нам не нужны повторяющиеся ящики. 
    # Уберем повторы
    result_list = delete_copy(result_list)
    
    print('\nAll (diffrent) e-mail addresses:')
    print(result_list)
    

Enter sitename, please
Please, wait a minute...
List of mails:
Process page  http://www.mosigra.ru/page/delivery/ ...........
Process page  http://www.mosigra.ru/customer/registration/ ...........
Process page  http://www.mosigra.ru/shop/kurskaya/ ...........
Process page  http://www.mosigra.ru/shop/tverskaya/ ...........
Process page  http://www.mosigra.ru/shop/taganskaya/ ...........
Process page  http://www.mosigra.ru/shop/prospekt_mira/ ...........
Process page  http://www.mosigra.ru/shop/novoslobodskaya/ ...........
Process page  http://www.mosigra.ru/shop/pavelec/ ...........
Process page  http://www.mosigra.ru/shop/belorusskaya/ ...........
Process page  http://www.mosigra.ru/shop/cvetnoi_bulvar/ ...........
Process page  http://www.mosigra.ru/allgames/ ...........
Process page  http://www.mosigra.ru/allgames/the_best/ ...........
Process page  http://www.mosigra.ru/allgames/big_company/ ...........
Process page  http://www.mosigra.ru/allgames/specialy_for_childre/ ...........
P